In [1]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm

In [2]:
class NeuralMatrixFactorization(nn.Module):
    def __init__(self, max_user, max_item, user_k=10, item_k=10, hidden_dim=50):
        super().__init__()
        self.user_emb = nn.Embedding(max_user, user_k, 0)
        self.item_emb = nn.Embedding(max_item, item_k, 0)
        self.mlp = nn.Sequential(
            nn.Linear(user_k + item_k, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim),
        )
    
    def forward(self, x):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)
        out = torch.cat([user_feature, item_feature], 1)
        out = self.mlp(out)
        out = nn.functional.sigmoid(out) * 5
        return out.squeeze()

In [6]:
import csv
from sklearn.feature_extraction.text import CountVectorizer

with open("data/ml-20m/movies.csv") as fp:
    reader = csv.DictReader(fp)
    def parse(d):
        movieId = int(d["movieId"])
        genres = d["genres"]
        return movieId, genres
    data = [parse(d) for d in reader]

movieIds = [x[0] for x in data]
genres = [x[1] for x in data]

# データに合わせてCountVectorizerを訓練する
cv = CountVectorizer(dtype="f4").fit(genres)
num_genres = len(cv.get_feature_names())

# keyがmovieIdでvalueがBoWのTensorのdictを作る
it = cv.transform(genres).toarray()
it = (torch.tensor(g, dtype=torch.float32) for g in it)
genre_dict = dict(zip(movieIds, it))

In [9]:
def first(xs):
    it = iter(xs)
    return next(it)

class MovieLensDataset(Dataset):
    def __init__(self, x, y, genres):
        assert len(x) == len(y)
        self.x = x
        self.y = y
        self.genres = genres
        
        # ジャンル辞書にないmovieIdの時のダミーデータ
        self.null_genre = torch.zeros_like(
        first(genres.values()))
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        # x = (userId, movieId)
        movieId = x[1]
        g = self.genres.get(movieId, self.null_genre)
        return x, y , g

In [10]:
from sklearn import model_selection
import pandas as pd

df = pd.read_csv("data/ml-20m/ratings.csv")
X = df [["userId", "movieId"]].values
Y = df[["rating"]].values

train_X, test_X, train_Y, test_Y = model_selection.train_test_split(X, Y, test_size=0.1)

train_dataset = MovieLensDataset(
    torch.tensor(train_X, dtype=torch.int64),
    torch.tensor(train_Y, dtype=torch.float32),
    genre_dict
)
test_dataset = MovieLensDataset(
    torch.tensor(test_X, dtype=torch.int64),
    torch.tensor(test_Y, dtype=torch.float32),
    genre_dict
)
train_loader = DataLoader(
    train_dataset, batch_size=1024, shuffle=True, num_workers=4
)
test_loader = DataLoader(
    test_dataset, batch_size=1024, num_workers=4
)

In [19]:
class NeuralMatrixFactorization2(nn.Module):
    def __init__(self, max_user, max_item, num_genres, user_k=10, item_k=10, hidden_dim=50):
        super().__init__()
        self.user_emb = nn.Embedding(max_user, user_k, 0)
        self.item_emb = nn.Embedding(max_item, item_k, 0)
        self.mlp = nn.Sequential(
            nn.Linear(user_k + item_k + num_genres, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)            
        )
    
    def forward(self, x, g):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)
        out = torch.cat([user_feature, item_feature, g], 1)
        out = self.mlp(out)
        out = nn.functional.sigmoid(out) * 5
        return out.squeeze()

In [22]:
def eval_net(net, loader, score_fn=nn.functional.l1_loss, device="cpu"):
    ys = []
    ypreds = []
    # loaderはジャンルのBoWも返す
    for x, y, g in loader:
        x = x.to(device)
        g = g.to(device)
        ys.append(y)
        # userId, movieId以外のジャンルのBoWもネットワーク関数に渡す
        with torch.no_grad():
            ypred = net(x, g).to("cpu")
        ypreds.append(ypred)
    score = score_fn(torch.cat(ys).squeeze(), torch.cat(ypreds))
    return score

In [24]:
from statistics import mean
max_user, max_item = X.max(0)
max_user = int(max_user)
max_item = int(max_item)

net = NeuralMatrixFactorization2(
    max_user+1, max_item+1, num_genres
)
opt = optim.Adam(net.parameters(), lr=0.01)
loss_f = nn.MSELoss()
net.to("cuda:0")
for epoch in range(5):
    loss_log = []
    net.train()
    for x, y, g in tqdm.tqdm(train_loader):
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        g = g.to("cuda:0")
        o = net(x, g)
        loss = loss_f(o, y.view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        loss_log.append(loss.item())
    net.eval()
    test_score = eval_net(net, test_loader, device="cuda:0")
    print(epoch, mean(loss_log), test_score.item(), flush=True)

  0%|          | 0/17579 [00:00<?, ?it/s]/home/melty0404/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 17579/17579 [03:10<00:00, 92.25it/s] 


0 0.7507812232787824 0.642629086971283


100%|██████████| 17579/17579 [03:13<00:00, 90.89it/s] 


1 0.6905302102027734 0.6320727467536926


100%|██████████| 17579/17579 [03:11<00:00, 91.67it/s] 


2 0.6675947502113879 0.628608226776123


100%|██████████| 17579/17579 [03:13<00:00, 90.73it/s] 


3 0.6523523629338975 0.62066251039505


100%|██████████| 17579/17579 [03:11<00:00, 91.90it/s] 


4 0.6364328650607951 0.6190357208251953


In [27]:
# 指定した位置だけ1で残りが0のTensorを返す補助関数
def make_genre_vector(i, max_len):
    g = torch.zeros(max_len)
    g[i] = 1
    return g

query_genres = [make_genre_vector(i, num_genres) for i in range(num_genres)]
query_genres = torch.stack(query_genres, 1)

# num_genres分だけuserId=100とmovieId=0のTensorを作って結合する
query = torch.stack([
    torch.empty(num_genres, dtype=torch.int64).fill_(100),
    torch.empty(num_genres, dtype=torch.int64).fill_(0)
], 1)

# GPUに転送
query_genres = query_genres.to("cuda:0")
query = query.to("cuda:0")

# スコアを計算
net(query, query_genres)

/home/melty0404/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([3.3780, 3.3790, 3.3784, 3.3790, 3.3777, 3.3762, 3.3782, 3.3777, 3.3779,
        3.3783, 3.3786, 3.3785, 3.3780, 3.3781, 3.3778, 3.3779, 3.3786, 3.3785,
        3.3775, 3.3779, 3.3786, 3.3787, 3.3768, 3.3782],
       device='cuda:0', grad_fn=<SqueezeBackward0>)